### Programa Calculos da Carteira B3

In [28]:
import pandas as pd
import os

from dados_pessoais import CAMINHO_ARQUIVO, NOME_ABA

#### Entrada e tratamento de dados

In [29]:
def carregar_dados():
    df_base = pd.read_excel(CAMINHO_ARQUIVO, sheet_name=NOME_ABA)
    return df_base

def tratar_dados(df_base):
    df_base = df_base.copy()

    # Renomear colunas para o padrão interno
    df_base = df_base.rename(columns={
        'Ticket': 'Código',
        'Posição': 'Quantidade',
        'Valor': 'Valor Total'
    })
    
    df_base['Código'] = df_base['Código'].str.strip().str.upper()
    df_base['C/V'] = df_base['C/V'].str.strip().str.upper()
    df_base['Data'] = pd.to_datetime(df_base['Data'], errors='coerce')
    
    df_base['Quantidade'] = df_base['Quantidade'].astype(int)
    df_base['Preço'] = df_base['Preço'].astype(float)
    df_base['Valor Total'] = df_base['Quantidade'] * df_base['Preço']
    
    return df_base





#### Calculo carteira FIFO

In [30]:
def calcular_fifo(df_base):
    df = df_base.sort_values('Data').copy()
    
    lotes = {}          # { 'PETR4': [ {'qtd': 100, 'preco': 25.30}, ... ] }
    registros_venda = []

    for _, row in df.iterrows():
        codigo = row['Código']
        tipo = row['C/V']
        qtd = row['Quantidade']
        preco = row['Preço']
        data = row['Data']
        valor_total = row['Valor Total']

        if codigo not in lotes:
            lotes[codigo] = []

        # COMPRA → cria novo lote
        if tipo == 'C':
            lotes[codigo].append({
                'quantidade': qtd,
                'preco': preco,
                'data': data
            })

        # VENDA → consome lotes FIFO
        elif tipo == 'V':
            qtd_restante = qtd
            custo_total = 0

            while qtd_restante > 0 and lotes[codigo]:
                lote = lotes[codigo][0]

                if lote['quantidade'] <= qtd_restante:
                    custo_total += lote['quantidade'] * lote['preco']
                    qtd_restante -= lote['quantidade']
                    lotes[codigo].pop(0)
                else:
                    custo_total += qtd_restante * lote['preco']
                    lote['quantidade'] -= qtd_restante
                    qtd_restante = 0

            resultado = valor_total - custo_total

            registros_venda.append({
                'Código': codigo,
                'Data': data,
                'Quantidade': qtd,
                'Valor_Venda': valor_total,
                'Custo_FIFO': custo_total,
                'Resultado': resultado
            })

    # Carteira remanescente por FIFO
    carteira = []
    for codigo, lista_lotes in lotes.items():
        qtd_total = sum(l['quantidade'] for l in lista_lotes)
        custo_total = sum(l['quantidade'] * l['preco'] for l in lista_lotes)
        preco_medio = custo_total / qtd_total if qtd_total > 0 else 0

        carteira.append({
            'Código': codigo,
            'Quantidade': qtd_total,
            'Custo_Total': custo_total,
            'Preço_Médio_FIFO': preco_medio
        })

    vendas_fifo = pd.DataFrame(registros_venda)
    carteira_fifo = pd.DataFrame(carteira)

    return vendas_fifo, carteira_fifo


#### Cálculo da Carteira

def calcular_carteira(df_base):
    compras = df_base[df_base['C/V'] == 'C']
    
    carteira = (
        compras
        .groupby('Código')
        .agg(
            Quantidade=('Quantidade', 'sum'),
            Custo_Total=('Valor Total', 'sum')
        )
    )
    
    carteira['Preço_Médio'] = carteira['Custo_Total'] / carteira['Quantidade']
    return carteira.reset_index()


### Ganhos e Perdas nas Operações

def calcular_ganho_perda(df_base, carteira):
    vendas = df_base[df_base['C/V'] == 'V'].copy()
    
    vendas = vendas.merge(
        carteira[['Código', 'Preço_Médio']],
        on='Código',
        how='left'
    )
    
    vendas['Custo'] = vendas['Quantidade'] * vendas['Preço_Médio']
    vendas['Resultado'] = vendas['Valor Total'] - vendas['Custo']
    
    return vendas


#### Consolidação Mensal

In [31]:
def consolidar_resultado_mensal(vendas):
    vendas['Mes'] = vendas['Data'].dt.to_period('M')
    
    mensal = (
        vendas
        .groupby('Mes')
        .agg(
            Total_Vendido=('Valor_Venda', 'sum'),
            Resultado=('Resultado', 'sum')
        )
        .reset_index()
    )
    
    return mensal


#### Exportação dos Resultados

In [32]:
from dados_pessoais import PASTA_BASE
import os

def exportar_relatorios(carteira, vendas, mensal):
    caminho_saida = os.path.join(PASTA_BASE, 'relatorio_carteira.xlsx')
    
    with pd.ExcelWriter(caminho_saida, engine='xlsxwriter') as writer:
        carteira.to_excel(writer, sheet_name='Carteira_Atual', index=False)
        vendas.to_excel(writer, sheet_name='Ganho_Perda_Vendas', index=False)
        mensal.to_excel(writer, sheet_name='Resultado_Mensal', index=False)
    
    print(f"Relatório gerado em: {caminho_saida}")


#### Pipeline

In [33]:
df_base = carregar_dados()
df_base = tratar_dados(df_base)

vendas_fifo, carteira_fifo = calcular_fifo(df_base)
mensal = consolidar_resultado_mensal(vendas_fifo)

exportar_relatorios(carteira_fifo, vendas_fifo, mensal)


Relatório gerado em: C:\Users\André Montagna\OneDrive\Pessoal\FINANÇAS\relatorio_carteira.xlsx
